In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator,os
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from contextlib import ExitStack

In [15]:
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')

In [16]:
tool = TavilySearchResults(max_results = 4)

In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver

stack = ExitStack()
memory = stack.enter_context(SqliteSaver.from_conn_string(":memory:"))

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>


In [20]:
from uuid import uuid4
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            merged.append(message)
    return merged
class AgentState(TypedDict):
    messages : Annotated[list[AnyMessage] , reduce_messages]

In [35]:
class Agent:
    def __init__(self,model,tools,system,checkpointer):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node('llm',self.call_llm)
        graph.add_node('action',self.take_action)
        graph.add_conditional_edges('llm',self.exist_action,{True:'action',False:END})
        graph.add_edge('action','llm')
        graph.set_entry_point('llm')
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before = ['action']
        )
        self.tools = {t.name : t for t in tools}
        self.model = model.bind_tools(tools)

    def call_llm(self, state : AgentState):
        messages = state['messages']
        if self.system :
            messages = [SystemMessage(content = self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages' : [message]}

    def take_action(self , state : AgentState):
        tool_calls = state ['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Action: {t}")
            if not t['name'] in self.tools:
                print("\n .....bad tool name .....")
                result = "bad tool name , retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to model !!")
        return {'messages' : results}

    def exist_action(self,state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    
        
    

In [36]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [37]:
model = ChatOpenAI(model = "gpt-3.5-turbo")
abot = Agent(model,[tool],system = prompt,checkpointer=memory)

In [38]:
messages = [HumanMessage(content="What is the weather in sf?")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
  for v in event.values():
      print(v)
while(abot.graph.get_state(thread).next):
    print('\n',abot.graph.get_state(thread),'\n')
    _input = input("proceed?(y/n)")
    if _input != 'y':
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)


Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_tcimQdoq1THBWWHMQGucgBTr', 'type': 'tool_call'}
Back to model !!


----------Streaning messages ----------

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [41]:
abot.graph.get_state(thread)
current_state = abot.graph.get_state(thread)
current_state.values['messages'][-1].tool_calls

Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_ggvc4r2vn3PyEsxIqQUMgLpx', 'type': 'tool_call'}
Back to model !!
Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_cgdbTxnCxeLc9Uq38lJdgvmt', 'type': 'tool_call'}
Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2024'}, 'id': 'call_VZJtH8mt0P6iTAijAIWqTWhH', 'type': 'tool_call'}
Back to model !!


------------update state message,this ill override the valuue in the state ---------------

In [42]:
_id = current_state.values['messages'][-1].tool_calls[0]['id']
current_state.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Louisiana'},
  'id': _id,
  'type': 'tool_call'}]


1. **Who won the Super Bowl in 2024?**

   The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime.

2. **In what state is the winning team's headquarters located?**

   The Kansas City Chiefs' headquarters is located in Missouri.

3. **What is the GDP of that state?**

   The GDP of Missouri as of 2024 is approximately $34.3 trillion.


In [ ]:
abot.graph.update_state(thread,current_state.values)

------After updating stream the message without interrup(by passing none as arg)--------

In [ ]:
abot.graph.get_state(thread)
for event in abot.graph.stream(None,thread):
    for v in event.values():
        print(v)
    

In [ ]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state,'\n..........\n')
    states.append(state)

-------- Time travelling to tird last state ----------------

In [ ]:
to_replay = states[-3]
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

--------update that state value ----------

In [ ]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Los Angeles,accuweather'},
  'id': _id,
  'type': 'tool_call'}]
branch_state=abot.graph.update_state(to_replay.config,to_replay.values)

In [ ]:
for event in abot.graph.stream(None,branch_state):
    for k, v in event.items():
        print(v)
    

---------- update the tome travelled state ,this upfate will add new message value rather than updationg existing one -----

In [ ]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
state_update= {"messages" : [ToolMessage(
    tool_call_id = _id,
    name = "tavily_search_results_json",
    content = "54 degree celcius"
)]}
branch_and_add = abot.graph.update_state(
    to_replay.config,
    state_update,
    as_node = 'action'
)
for event in abot.graph.stream(None,branch_and_add):
    for v in event.values():
        print(v)